# Demo notebook for Jupyter - openBIS integration
This notebook demonstrates how we can connect to the openBIS server, download some data, analyze the data and then save the results in openBIS.

First, use the `configure openBIS connection` button in the toolbar, to connect to your openBIS server. When the connection has been selected, download the dataset in experiment `/SIS_HLUETCKE/DEMO/JUPYTER-DEMO` via the `Download openBIS dataset` button.

### Import required modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.morphology import reconstruction, remove_small_objects
from skimage.measure import label, regionprops

%matplotlib inline

### Read and display the data

In [ ]:
img = imread('./20190328133056219-504/original/blobs.tif')

In [ ]:
fig = plt.figure(figsize=(10,5))
fig.add_subplot(121)
plt.imshow(img, cmap='gray')
plt.title('blobs')
fig.add_subplot(122)
plt.hist(img.ravel(), bins=10)
plt.title('Histogram of blobs')
plt.show()

### Process the data
Let's say our analysis task is to segement the white blobs from the dark background. We will first set a threshold to binarize the image. Then we perform some further image processing to refine the results. Finally, we label the segmented blobs and measure their size. 

In [ ]:
intensity_threshold = 150
binary = img > intensity_threshold

In [ ]:
plt.imshow(binary, cmap='gray')
plt.title('Binary image')

Fill the small holes in the white blobs.

In [ ]:
seed = np.copy(binary)
seed[1:-1, 1:-1] = binary.max()
filled = reconstruction(seed, binary, method='erosion')

In [ ]:
plt.imshow(filled, cmap='gray')
plt.title('Filled image')

Remove some small, spurious blobs.

In [ ]:
removed_small = remove_small_objects(filled.astype(bool), min_size=20)

In [ ]:
plt.imshow(removed_small, cmap='gray')
plt.title('Small objects removed')

Label the objects found.

In [ ]:
label_img = label(removed_small, connectivity=img.ndim)

In [ ]:
plt.imshow(label_img*10, cmap='prism')
plt.title('Labeled image')

Get properties of the detected objects.

In [ ]:
props = regionprops(label_img)
area = np.array([x.area for x in props])

In [ ]:
print('Found %d blobs.' % (len(area)))
print('Area min: %1.0f pixels' % (np.min(area)))
print('Area max: %1.0f pixels' % (np.max(area)))
print('Area mean: %1.0f pixels' % (np.mean(area)))

Now we can upload the notebook back to openBIS using the `Upload notebook and data` button in the toolbar. Make sure 
to save the notebook first: `File --> Save and checkpoint`
It is also possible to create static versions of the notebook: `File --> Download as --> HTML`.